# Analysis of Seasonal Mortality in Python

Bernhard Klinger <klinger@btinternet.com>

## Summary

In common with other countries more people in England & Wales die in winter than during other parts of the year. In addition, the number of excess winter deaths has historically varied significantly between years. 

This paper outlines out how Python libraries could be used to analyse and explain the seasonal mortality pattern. 

The regression model developed indicates that cold weather and flu vaccine effectiveness could explain most of the variation in excess winter mortality from 2011 to 2019.  

## Data


### Data source

#### Actual mortality and population data 

The sample dataset provided has been derived from provisional weekly death counts and regional population estimates for 2010 to 2019 as published on the website of the Office of National Statistics (ONS). As the seasonal mortality effects are most pronounced for older ages, we have limited the study to the population in England & Wales aged 65 years and older.

#### Winter mortality

We assume that mortality would be elevated in the period from December to April. December to February fall within the winter quarter and March to April within the spring quarter in our analysis. 

#### Air frost days

The number of air frost (AF) days has been selected to reflect the impact of cold weather on mortality. This metric could be a better indicator for cold waves than monthly mean temperature. Monthly data points for England and Wales have been sourced from the [Met Office](https://www.metoffice.gov.uk/research/climate/maps-and-data/uk-and-regional-series) website.

#### Flu vaccine effectiveness 

Influenza related deaths are a significant contributor to winter mortality for older age groups. The effectiveness of flu vaccine in a particular year could therefore help to explain variations in excess mortality during the flu season. In our analysis, we have used a single data point per calendar year, the effectiveness of seasonal flu vaccines for all age groups. This data item was sourced from the [Center of Disease Control](https://www.cdc.gov/flu/vaccines-work/effectiveness-studies.htm) website for 2009/2010 to 2014/2015 and from [Public Health England (PHE)](https://www.gov.uk/government/publications/influenza-vaccine-effectiveness-seasonal-estimates) publications for 2015/2016 to date. 

### Data structure 



|Column |Description |
| :--- | :--- |
| Deaths | Average number of weekly deaths |
| Persons | Estimated average number of lives (adjusted for changes in the age profile within each age group over time) |
| Persons_NoAdj | Estimated average number of lives without adjustments |
| Rate| Average annualised weekly mortality rate per 1000 lives (adjusted) |
| Rate_NoAdj | Average annualised weekly mortality rate per 1000 lives (unadjusted) |
| Year | Indicator to estimate improvement trend: 2010 == 0, 2019 == 9 |
| Winter | Indicator for months with elevated mortality (i.e. all of winter quarter and 2/3 of the spring quarter) |
| AF | Number of air frost days in the quarter |
| VE |  Flu vaccine effectiveness percentage |
| Age=Mnn | Indicator for each male age group |
| Age=Fnn | Indicator for each female age group |


The index values represent the last day of each quarter. 

### Licence

The data sourced from ONS, Met Office and PHE contains public sector information licensed under the [Open Government Licence](http://www.nationalarchives.gov.uk/doc/open-government-licence). 

### Limitations

This paper is a contribution to demonstrate how Python libraries could be used in the study of excess winter mortality. The model results should be viewed as illustrative and proof of concept for a more in depth study.  

## Analysis

Let's get started, load the dataset and look at some statistics.

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12.5, 6.0)
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
sns.set()

mydata = pd.read_pickle("MortalityData_2010to2019")
mydata.describe()

We would expect the predictors to impact mortality quite differently for each age group. To capture the dependency between age and the other predictive features, we need to add interaction terms. 

In [ ]:
# Define the age groups and descriptions

agelist = ["65","75","85"]
agedesc = ["65-74","75-84","85+  ","All - Test data only"]

for i in range(0,3):
    age=agelist[i]
    age_col = mydata["Age=M"+age]+mydata["Age=F"+age]
    mydata["WINT_"+age]=mydata["Winter"]*age_col
    mydata["IMP_"+age]=mydata["Year"]*age_col
    
for i in range(0,3):
    age=agelist[i]
    age_col = mydata["Age=M"+age]+mydata["Age=F"+age]
    mydata["AF_"+age]=mydata["AF"]*age_col
    mydata["VE_"+age]=mydata["VE"]*age_col
      
X=mydata.copy() # We will store both the target and the predictors in one dataframe to simplify creation of our plots

# Remove features no longer required and add placeholders for predicted values

X.drop(columns=["VE","Year","Winter","AF"],inplace=True)
X.insert(5,"PredR",0)
X.insert(6,"PredD",0)
X.insert(7,"PredR_Base",0)
X.insert(8,"PredD_Base",0)

Next, we fit a linear regression model. To assess the explanatory effect of our predicators, we fit a "base" model that excludes the AF and VE features.

In [ ]:
X_pred= X.iloc[:,9:] # predictors
y= X["Rate"] # target
X_train, X_test, y_train, y_test = train_test_split(X_pred, y, test_size=0.2, random_state=10101)

# Fit full model with all predictive features, i.e. frost days and vaccine effectiveness

model=LinearRegression()       
model.fit(X_train, y_train) 

X["PredR"]= model.predict(X_pred)
X["PredD"]= mydata['Persons']*X["PredR"]/1000/52

# Calculate the contribution from each feature, i.e. feature coefficient  x value of the feature
X_cont= X_pred.mul(model.coef_,axis=1)
X_cont= X_cont.mul(list(mydata["Persons"]),axis=0)/1000/52


# Fit base model with no predictive features, i.e. Age group, winter month indictor and annual improvement trend only
# Use same train/test split as for the full model

model_base = LinearRegression()
model_base.fit(X_train.iloc[:,:12], y_train)

X["PredR_Base"]= model_base.predict(X_pred.iloc[:,:12])
X["PredD_Base"]= mydata['Persons']*X["PredR_Base"]/1000/52

# Perform cross validation for both models by age group
for i in range(0,3):
        age = agelist[i]
        Xg=X[(X["Age=M"+age] == 1) | (X["Age=F"+age]==1)]
        print("Full model CV Score - Age " + agedesc[i]+": ", ["%0.4f" % val for val in \
                                                               cross_val_score(LinearRegression(),Xg.iloc[:,7:],Xg["Rate"], cv=5)])
        print("Base model CV Score - Age " + agedesc[i]+": ",["%0.4f" % val for val in \
                                                              cross_val_score(LinearRegression(),Xg.iloc[:,7:19],Xg["Rate"], cv=5)],"\n")

At older ages, mortality rates vary quite significantly between each age group. That's why we will analyse the model for each age group separately. 

Cross validation scores look good. Let's prepare a few dataframes to feed into our plots ...

In [ ]:
dfd = [{},{},{},{},{},{},{},{},{},{}] # list of dictionaries for our plots

for i in range(0,4): 
    if i < 3:
        age = agelist[i]
        Xg=X[(X["Age=M"+age] == 1) | (X["Age=F"+age]==1)] # Apply the age group filter
        Xg= Xg.groupby(Xg.index).sum()
        dfd[0].update({"Actual - Age "+agedesc[i]:Xg["Deaths"]})
        dfd[1].update({"Unexplained - Age "+agedesc[i]: Xg["Deaths"]-Xg["PredD"]})
        
        Xgw= Xg[Xg.index.month == 2] # Apply winter quarter filter
        Xgw= Xgw.groupby(Xgw.index).sum()
        dfd[2].update({"Actual - Age "+agedesc[i]:Xgw['Deaths'] - Xgw['PredD_Base']})
        dfd[3].update({"Unexplained - Age "+agedesc[i]: Xgw['Deaths']-Xgw['PredD']})
        
        Xgc=X_cont[X_cont.index.month == 2] # Apply winter quarter filter
        Xgc= Xgc.groupby(Xgc.index).sum()
        dfd[4].update({"Contribution - AF "+agedesc[i]:Xgc['AF_'+age]})
        dfd[5].update({"Contribution - VE "+agedesc[i]:Xgc['VE_'+age]})
        
    else: 
        age = "All"
        Xg=X.groupby(X.index).sum()
    dfd[i+6]= {"Actual - Age "+agedesc[i]:Xg['Deaths']/Xg['Persons']*1000*52,"Predicted - Age "+agedesc[i]: Xg['PredD']/Xg['Persons']*1000*52}   

... and run the first set of plots to analyse the model.

In [ ]:
# Analysis of actual and predicted seasonal deaths

for i in range(0,4):
    plot_df= pd.DataFrame(dfd[i])
    ax = plot_df.plot(kind="bar",alpha=1,stacked=True, rot=0)
    as_settitle="Seasonal variation of deaths by age group"
    if i < 2: 
        ax.set_title("Seasonal variation of total deaths by age group",fontsize=14)
        ax.set_xticklabels(map(lambda x : x.year if x.month < 3 else "",plot_df.index))
    else:
        ax.set_ylim(-1000,1300)
        ax.set_title("Variation in winter quarter excess deaths by age group",fontsize=14)
        ax.set_xticklabels(map(lambda x : x.year,plot_df.index))

We can see that the model performs pretty well to explain the seasonal variations. Specifically, for the years where we observe the largest variance from the average number of winter deaths (2014, 2015 and 2018), the model explains most of the variation.

Try out different seeds and test data sizes to investigate how this might impact the model fit.

Let's have a look how the AF and VE features contribute to explain the variation in the winter excess mortality.

In [ ]:
for i in range(4,6):
    plot_df= pd.DataFrame(dfd[i])
    ax = plot_df.plot(kind="bar",alpha=1,stacked=True, rot=0)
    if i ==4: 
        ax.set_ylim(-100,2000)
        ax.set_title("Modelled contribution of cold weather to winter quarter excess deaths",fontsize=14)
    else: 
        ax.set_ylim(-2000,100)
        ax.set_title("Modelled contribution of flu vaccine effectiveness to winter quarter excess deaths",fontsize=14)
    ax.set_xticklabels(map(lambda x : x.year,plot_df.index))

2014 was the mildest winter in our analysis period with average flu vaccine effectiveness, resulting in a low number of winter deaths. In 2015 and 2018, the winters were relatively cold with low flu vaccine effectiveness, resulting in a much higher number of winter deaths.

Why don't you try and analyse how the model could explain variations in deaths during the spring quarter?

Next, we will  use our model to visualise the variation in mortality rates - these are expressed as the number of annualised deaths per 1000 lives in each age group.  

In [ ]:
# Analysis of actual and predicted seasonal mortality rates 
    
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.suptitle("\nMortality rate - Actual and predicted by age group")
fig.subplots_adjust(hspace=0.3,top=.9)
for i in range(0,4):
    plot_df= pd.DataFrame(dfd[i+6])
    plot_df=plot_df.resample("M").interpolate(method='cubic')
    plot = plot_df.plot(kind='line',alpha=1, figsize=(15, 9),linewidth=2,ax=axes[i//2][i%2])

Our model seems to perform well to explain the variations in winter mortality rates. It doesn't include data that could help to explain the variance in summer mortality (e.g. the drop in 2013). Can you think of a suitable predictor?

The annual rate of mortality improvement appears to be  more pronounced for the ages below 85 compared to the 85+ age group. Care needs to be taken to adjust the age groups for the variation in the age profile within each group over time. Think about the baby boomer generation which would have moved through the 65-74 age group over the period. We have adjusted the population data for this effect at a high-level. 

Try to fit a model based on the unadjusted population data - can you see the impact on the model fit for the 65-74 age group?   

Finally, we will have a look at the residual plots from the seaborn library. These are an alternative option to visualise the fit of our regression model.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(16,9))
fig.suptitle("\nMortality rate - Residuals by age group")
fig.subplots_adjust(hspace=0.5,top=0.9)

for i in range(0,4):
    if i < 3:
        filter = (X["Age=M"+agelist[i]] == 1) | (X["Age=F"+agelist[i]]==1)
        X1 = X[filter]
    else: X1=X
    scr = r2_score(X1["Rate"],X1["PredR"])
    plottitle = "Age: "+agedesc[i]+", $R^2$= "+str("%.4f"%scr)
    ax1 = sns.residplot(X1["Rate"],X1["PredR"],ax=axes[i//2,i%2])
    ax1.set_title(plottitle) 

In conclusion, our model indicates that cold weather and the effectiveness of the annual flu vaccine could be used to explain the variations in seasonal mortality.  